# Imports

In [ ]:
from flipside import Flipside
from config import flipside_api_key, flipside_nft_holder_address

In [ ]:
import pandas as pd
import os
from datetime import datetime as dt
from datetime import timedelta


In [ ]:
from app.data.local_storage import (
    read_json,
    read_csv
)


In [ ]:
sdk = Flipside(flipside_api_key)

# Data Checklist

```markdown
* [X] Curve Locks
* [X] Curve Votes

* [X] Gauge to LP map
* [] Curve Liquidity

* [X] Convex Snapshot
* [X] Votium Bounties

* [X] StakeDAO Snapshot
```

# Define Save Function

In [ ]:
data_path = "/app/data/source"

def get_cwd():
    cwd_temp = os.getcwd()
    temp_split = cwd_temp.split('/')
    cwd = ""
    for x in temp_split:
        if x == 'experiments':
            break
        elif x == '':
            pass
        else:
            cwd += "/"+x       
    return cwd

def print_metrics(query):
    started_at = query.run_stats.started_at
    ended_at = query.run_stats.ended_at
    elapsed_seconds = query.run_stats.elapsed_seconds
    record_count = query.run_stats.record_count
    print(f"This query took ${elapsed_seconds} seconds to run and returned {record_count} records from the database.")


def query_and_save(_query, _filename, _df_base = [], _page_size = 5000):
    try:
        # Initial Query
        if len(_df_base) > 0:
            print("based")
            df_output = _df_base
        else:
            df_output = []

        print(f"___\n{_filename}")
        print(f"querying page: 1")
        query_result_set = sdk.query(_query, page_size = _page_size)
        if len(df_output) == 0:
            df_output = pd.json_normalize(query_result_set.records)
        else:
            # Concat Dataframes
            df_local = pd.json_normalize(query_result_set.records)
            df_output = pd.concat([df_output, df_local], ignore_index=True)

        # Metrics
        print_metrics(query_result_set)
        i = 2
            
        # Handle Pagination
        if len(query_result_set.records) >= _page_size:
            keep_going = True
            while keep_going:
                print(f"querying page: {i}")
                extended_result_set = sdk.get_query_results(
                    query_result_set.query_id,
                    page_number=i,
                    page_size=_page_size
                )
                # Metrics
                print_metrics(query_result_set)
                
                # Concat Dataframes
                df_local = pd.json_normalize(extended_result_set.records)
                df_output = pd.concat([df_output, df_local], ignore_index=True)
                
                # Check if continue
                print(len(extended_result_set.records) < _page_size)
                print(len(extended_result_set.records))
                if len(extended_result_set.records) < _page_size:
                    keep_going = False
                i += 1
        # Save
        cwd = get_cwd()
        full_filename = cwd+ data_path + '/' + filename+'.csv'
        df_output.to_csv(full_filename) 
    except Exception as e:
        print(e)
        if len(_df_base) > 0:
            df_output = _df_base
        else:
            df_output = []

    return df_output

In [ ]:
def get_df_and_target(filename, target = 'block_timestamp'):
    # gets dataframe and max value of target intended for time values
    resp_dict = read_csv(filename, 'source')
    df = pd.json_normalize(resp_dict)
    print(df.keys())

    # Get Max Value of target
    temp_df = df.sort_values(target).tail(1)
    search_result = temp_df.iloc[0][target] 
    # # If date only drown out time diffs by removing last
    # if target == 'date_day':
    #     # remove most current
    #     df = df[df[target] < search_result]
    #     # find new max value of target
    #     temp_df = df.sort_values(target).tail(1)
    #     search_result = temp_df.iloc[0][target] 

    try:
        if 'T' in search_result:
            split = search_result.split("T")
            search_result = split[0]+" "+split[1][:-1]
    except:
        pass
    print(search_result)
    return df, search_result


# SQL

## Curve Locker

In [ ]:
filename = 'curve_locker'

In [ ]:
df, block_timestamp = get_df_and_target(filename)

In [ ]:
curve_locker_address = '0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2'

curve_locker_query = f"""SELECT 
  *,
  WEEK(BLOCK_TIMESTAMP) as WEEK_NUMBER,
  DAYOFWEEK(BLOCK_TIMESTAMP) as WEEK_DAY

FROM ethereum.core.ez_decoded_event_logs
WHERE CONTRACT_ADDRESS = lower('{curve_locker_address}')
AND BLOCK_TIMESTAMP >'{block_timestamp}'

"""


In [ ]:
df_curve_locker = query_and_save(curve_locker_query, filename, df)

## Curve Gauge Votes

In [ ]:
filename = 'curve_gauge_votes'

In [ ]:
df, block_timstamp = get_df_and_target(filename)

In [ ]:
curve_voter_address = '0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB'

curve_gauge_vote_query = f"""SELECT 
    SYMBOL, 
    NAME, 
    WEEK(BLOCK_TIMESTAMP) as WEEK_NUMBER,
    DAYOFWEEK(BLOCK_TIMESTAMP) as WEEK_DAY,
    DECODED_LOG,
    TX_HASH,
    BLOCK_TIMESTAMP

FROM ethereum.core.ez_decoded_event_logs LOGS
 LEFT JOIN ethereum.core.dim_contracts CONTRACT
   ON CONTRACT.address = lower(LOGS.DECODED_LOG:gauge_addr::string)
WHERE CONTRACT_ADDRESS = lower('{curve_voter_address}')
AND EVENT_NAME = 'VoteForGauge'
AND BLOCK_TIMESTAMP >'{block_timestamp}'
ORDER BY BLOCK_TIMESTAMP ASC
"""

In [ ]:
df_curve_guage_votes = query_and_save(curve_gauge_vote_query, filename, df, 5000)

## Snapshot Votes

In [ ]:
filename = 'convex_snapshot_votes'
df, vote_timestamp = get_df_and_target(filename, 'vote_timestamp')

In [ ]:
convex_snapshot_votes_query = f"""
SELECT 
    PROPOSAL_ID, 
    PROPOSAL_START_TIME, 
    PROPOSAL_END_TIME, 
    PROPOSAL_TITLE, 
    PROPOSAL_AUTHOR,
    VOTE_OPTION,
    VOTING_POWER,
    VOTE_TIMESTAMP,
    QUORUM,
    CHOICES,
    VOTING_PERIOD,
    NETWORK,
    SPACE_ID,
    VOTER,
    ADDRESS_NAME,
    LABEL_TYPE, 
    LABEL_SUBTYPE,
    LABEL
FROM external.snapshot.ez_snapshot as SNAPSHOT
 LEFT JOIN ethereum.core.dim_labels LABELS
   ON SNAPSHOT.VOTER = LABELS.ADDRESS
WHERE SPACE_ID = 'cvx.eth' 
AND PROPOSAL_TITLE LIKE 'Gauge Weight%'
AND VOTE_TIMESTAMP > '{vote_timestamp}'
"""

In [ ]:
df_snapshot_votes = query_and_save(convex_snapshot_votes_query, filename, df)

In [ ]:
filename = 'stakedao_snapshot'
df, vote_timestamp = get_df_and_target(filename, 'vote_timestamp')

In [ ]:
stakedao_snapshot_votes_query = f"""
SELECT 
    PROPOSAL_ID, 
    PROPOSAL_START_TIME, 
    PROPOSAL_END_TIME, 
    PROPOSAL_TITLE, 
    PROPOSAL_AUTHOR,
    VOTE_OPTION,
    VOTING_POWER,
    VOTE_TIMESTAMP,
    QUORUM,
    CHOICES,
    VOTING_PERIOD,
    NETWORK,
    SPACE_ID,
    VOTER,
    ADDRESS_NAME,
    LABEL_TYPE, 
    LABEL_SUBTYPE,
    LABEL
FROM external.snapshot.ez_snapshot as SNAPSHOT
 LEFT JOIN ethereum.core.dim_labels LABELS
   ON SNAPSHOT.VOTER = LABELS.ADDRESS
WHERE SPACE_ID = 'sdcrv.eth' 
AND (PROPOSAL_TITLE LIKE 'Gauge vote - CRV%'
OR
PROPOSAL_TITLE LIKE 'Gauge vote CRV%'
)
AND VOTE_TIMESTAMP > '{vote_timestamp}'

"""

In [ ]:
df_snapshot_votes = query_and_save(stakedao_snapshot_votes_query, filename, df,  1500)

In [ ]:
df_snapshot_votes

# Votium Bounties

In [ ]:
filename = 'votium_bounty_for_round'
df, block_timestamp = get_df_and_target(filename)

In [ ]:
votium_bounty_query = f"""
SELECT 
  EVENT_NAME,
  TX_HASH,
  DECODED_LOG:_choiceIndex::int as choice_index,
  DECODED_LOG:_amount::int / pow(10,18) as amount,
  DECODED_LOG:_proposal::string as proposal_id,
  DECODED_LOG:_token::string as bounty_token_address,
  ORIGIN_FROM_ADDRESS,
  BLOCK_TIMESTAMP as block_timestamp,
  PRICE as price,
  price * amount as bounty_value,
  contracts.NAME as token_name,
  contracts.SYMBOL as token_symbol
FROM ethereum.core.ez_decoded_event_logs as logs

LEFT JOIN ethereum.core.fact_hourly_token_prices as prices
ON logs.DECODED_LOG:_token = prices.token_address
AND time_slice(logs.BLOCK_TIMESTAMP::timestamp_ntz, 1, 'HOUR') = prices.HOUR

LEFT JOIN ethereum.core.dim_contracts as contracts
ON logs.DECODED_LOG:_token = contracts.ADDRESS

WHERE CONTRACT_ADDRESS= lower('0x19bbc3463dd8d07f55438014b021fb457ebd4595')

AND EVENT_NAME = 'Bribed'
AND BLOCK_TIMESTAMP > '{block_timestamp}'


ORDER BY block_timestamp
"""

In [ ]:
df_votium_bounty = query_and_save(votium_bounty_query, filename, df)

# Reference: Gauge to LP Map

In [ ]:
filename = 'gauge_to_lp_map'
df, block_timestamp = get_df_and_target(filename, 'deployed_timestamp')


In [ ]:
curve_voter_address = '0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB'

gauge_to_lp_map_query = f"""
-- 0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB
with gauge_types as (
  SELECT 
    DECODED_LOG:name::string as name,
    DECODED_LOG:type_id::int as type_id,
    BLOCK_TIMESTAMP as block_timestamp,
    TX_HASH as tx_hash
  
  FROM ethereum.core.ez_decoded_event_logs LOGS
   LEFT JOIN ethereum.core.dim_contracts CONTRACT
     ON CONTRACT.address = lower(LOGS.DECODED_LOG:gauge_addr::string)
  WHERE CONTRACT_ADDRESS = lower('{curve_voter_address}')
  AND EVENT_NAME = 'AddType'
),

new_type_weight as (
  SELECT 
    ifnull(DECODED_LOG:time::int, 0) as time,
    DECODED_LOG:total_weight::string as total_weight,
    ifnull(DECODED_LOG:type_id::int, 0) as type_id,
    DECODED_LOG:weight::string as weight,
    TX_HASH,
    BLOCK_TIMESTAMP
  
  FROM ethereum.core.ez_decoded_event_logs
  WHERE CONTRACT_ADDRESS = lower('{curve_voter_address}')
  AND EVENT_NAME = 'NewTypeWeight'
  ORDER BY BLOCK_TIMESTAMP ASC

),

new_gauges as (

  SELECT 
      SYMBOL, 
      NAME, 
      DECODED_LOG:addr::string as gauge_addr,
      DECODED_LOG:gauge_type::int as gauge_type,
      DECODED_LOG:weight::int as weight,
      TX_HASH,
      BLOCK_TIMESTAMP
  
  FROM ethereum.core.ez_decoded_event_logs LOGS
   LEFT JOIN ethereum.core.dim_contracts CONTRACT
     ON CONTRACT.address = lower(LOGS.DECODED_LOG:addr::string)
  WHERE CONTRACT_ADDRESS = lower('{curve_voter_address}')
  AND EVENT_NAME = 'NewGauge'
  ORDER BY BLOCK_TIMESTAMP ASC
  
),

gauge_meta as (
  SELECT 
    new_gauges.gauge_type as type_id,
    gauge_types.name as type_name,
    new_gauges.name,
    new_gauges.symbol,
    new_gauges.gauge_addr,
    new_gauges.weight,
    new_type_weight.weight as type_weight,
    new_type_weight.total_weight as type_total_weight,
    new_type_weight.time as type_weight_time,
    new_gauges.tx_hash,
    new_gauges.block_timestamp as vote_timestamp
  FROM new_gauges 
  LEFT JOIN gauge_types
  ON new_gauges.gauge_type = gauge_types.type_id
  LEFT JOIN new_type_weight
  ON new_gauges.gauge_type = new_type_weight.type_id
  ORDER BY new_gauges.block_timestamp DESC
),


-- BREAK between meta and deployers

v2_deployer as (
  SELECT
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    DECODED_LOG:token::string as token_addr,
    'v2' as source
      
  FROM ethereum.core.ez_decoded_event_logs
  WHERE CONTRACT_ADDRESS = lower('0xF18056Bbd320E96A48e3Fbf8bC061322531aac99') -- v2 deployer
  AND EVENT_NAME = 'LiquidityGaugeDeployed'
  AND BLOCK_TIMESTAMP > '{block_timestamp}'


),

factory_deployer as (
  SELECT 
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    '' as token_addr,
    'factory' as source

  FROM ethereum.core.ez_decoded_event_logs as logs

  WHERE logs.CONTRACT_ADDRESS = lower('0xB9fC157394Af804a3578134A6585C0dc9cc990d4')  -- factory
  AND logs.EVENT_NAME = 'LiquidityGaugeDeployed'
  AND BLOCK_TIMESTAMP > '{block_timestamp}'

),


stable_deployer as (
  SELECT 
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    '' as token_addr,
    'stable_factory' as source

  FROM ethereum.core.ez_decoded_event_logs as logs

  WHERE logs.CONTRACT_ADDRESS = lower('0x4F8846Ae9380B90d2E71D5e3D042dff3E7ebb40d')  -- stable_factory
  AND logs.EVENT_NAME = 'LiquidityGaugeDeployed'
  AND BLOCK_TIMESTAMP > '{block_timestamp}'

),


multichain_deployer as (
  SELECT 
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:_gauge::string as gauge_addr,
    DECODED_LOG:_chain_id::string as chain_id,
    DECODED_LOG:_deployer::string as deployer,
    DECODED_LOG:_implementation::string as implementation,
    DECODED_LOG:_salt::string as salt,
    '' as pool_addr,
    '' as token_addr,
    'multichain_factory' as source

  FROM ethereum.core.ez_decoded_event_logs as logs

  WHERE logs.CONTRACT_ADDRESS = lower('0xabc000d88f23bb45525e447528dbf656a9d55bf5')  -- multichain_factory
  AND logs.EVENT_NAME = 'DeployedGauge'
  AND BLOCK_TIMESTAMP > '{block_timestamp}'

),

tricrypto_deployer as (
  SELECT 
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    '' as token_addr,
    'tricrypto_factory' as source

  FROM ethereum.core.ez_decoded_event_logs as logs

  WHERE logs.CONTRACT_ADDRESS = lower('0x0c0e5f2ff0ff18a3be9b835635039256dc4b4963')  -- tricrypto_factory
  AND logs.EVENT_NAME = 'LiquidityGaugeDeployed'
  AND BLOCK_TIMESTAMP > '{block_timestamp}'
),


combo as (
  SELECT gauge_addr, pool_addr, token_addr, source, block_timestamp, '' as chain_id FROM v2_deployer
  UNION
  SELECT gauge_addr, pool_addr, token_addr, source, block_timestamp,'' as chain_id   FROM factory_deployer
  UNION
  SELECT gauge_addr, pool_addr, token_addr, source, block_timestamp,'' as chain_id FROM stable_deployer
  UNION
  SELECT gauge_addr, pool_addr, token_addr, source, block_timestamp,'' as chain_id FROM tricrypto_deployer
  UNION
  SELECT gauge_addr, pool_addr, token_addr, source, block_timestamp, chain_id  FROM multichain_deployer

),

deployer_meta as (
  SELECT 
    combo.gauge_addr as gauge_addr, 
    combo.pool_addr as pool_addr, 
    combo.token_addr as token_addr, 
    combo.source as source, 
    combo.chain_id as chain_id,
    combo.block_timestamp as block_timestamp,
    contracts.NAME as gauge_name,
    contracts.SYMBOL as gauge_symbol,
    pool_contracts.NAME as pool_name,
    pool_contracts.SYMBOL as pool_symbol,
    token_contracts.NAME as token_name,
    token_contracts.SYMBOL as token_symbol
  FROM combo
  LEFT JOIN ethereum.core.dim_contracts as contracts
    ON combo.gauge_addr = contracts.ADDRESS
  LEFT JOIN ethereum.core.dim_contracts as pool_contracts
    ON combo.pool_addr = pool_contracts.ADDRESS
  LEFT JOIN ethereum.core.dim_contracts as token_contracts
    ON combo.token_addr = token_contracts.ADDRESS
)


SELECT 
    gauge_meta.type_id,
    gauge_meta.type_name,
    IFNULL(gauge_meta.name, deployer_meta.gauge_name) as name,
    IFNULL(gauge_meta.symbol, deployer_meta.gauge_symbol) as symbol,
    IFNULL(gauge_meta.gauge_addr, deployer_meta.gauge_addr) as gauge_addr,
    gauge_meta.weight,
    gauge_meta.type_weight,
    gauge_meta.type_total_weight,
    gauge_meta.type_weight_time,
    gauge_meta.tx_hash,
    gauge_meta.vote_timestamp,

    deployer_meta.pool_addr, 
    deployer_meta.token_addr, 
    deployer_meta.source, 
    deployer_meta.block_timestamp as deployed_timestamp,
    deployer_meta.gauge_name,
    deployer_meta.gauge_symbol,
    deployer_meta.pool_name,
    deployer_meta.pool_symbol,
    deployer_meta.token_name,
    deployer_meta.token_symbol,
    deployer_meta.chain_id
FROM gauge_meta
FULL JOIN deployer_meta
ON gauge_meta.gauge_addr = deployer_meta.gauge_addr
ORDER BY deployed_timestamp DESC
"""

In [ ]:
df_curve_gauge_map = query_and_save(gauge_to_lp_map_query, filename, df)

# Curve Liquidity

In [ ]:
filename = 'liquidity_general'
df, date_day = get_df_and_target(filename, 'date_day')

In [ ]:
# date_day.date() + timedelta(days=1)
# tomorow_date = dt.strptime(date_day,'%Y-%m-%d %H:%M:%S.%f') + timedelta(days=1)

# print(date_day)
# print(tomorow_date)

df = df[df['date_day'] < date_day]
df 


In [ ]:
start_date = '2023-03-01 00:00:00.000'
curve_liquidity_query = f"""
with curve_swaps as (
SELECT
*
FROM ethereum.core.ez_dex_swaps
WHERE PLATFORM = 'curve' 
),

tokens_out as (
SELECT 
DISTINCT TOKEN_OUT as token,
CONTRACT_ADDRESS as pool_address,
POOL_NAME as pool_name,
SYMBOL_OUT as symbol
FROM curve_swaps
GROUP BY TOKEN_OUT, SYMBOL_OUT, CONTRACT_ADDRESS, POOL_NAME
),

tokens_in as (
SELECT 
DISTINCT TOKEN_IN as token,
CONTRACT_ADDRESS as pool_address,
POOL_NAME as pool_name,

SYMBOL_IN  as symbol
FROM curve_swaps
-- GROUP BY TOKEN_IN
),

pools as (
SELECT
DISTINCT CONTRACT_ADDRESS as pool_address,
POOL_NAME as pool_name,
'ETH' as symbol,
'' as token
FROM curve_swaps
-- GROUP BY CONTRACT_ADDRESS

),

tokens as (
select 
*
FROM tokens_out
UNION all
select 
*
FROM tokens_in
UNION all
select 
token, pool_address, pool_name, symbol
FROM pools
),

all_tokens as (
select 
* 
from tokens
group by token, symbol, pool_address, pool_name
),


-- COMBINE TOKENS WITH DATES
dates as (
select 
date_day 
from ethereum.core.dim_dates 
where date_day between '{date_day}' and current_date()
),

dates_x_tokens as (
select 
date_day,
symbol,
token,
pool_address,
pool_name

from dates
cross join all_tokens
ORDER BY date_day DESC
),


dates_x_tokens_x_price as (
select 
dates_x_tokens.date_day,
dates_x_tokens.symbol,
dates_x_tokens.token,
pool_address,
pool_name,
AVG(prices.PRICE) as daily_price
FROM dates_x_tokens
LEFT JOIN ethereum.core.fact_hourly_token_prices as prices
ON (dates_x_tokens.token = prices.TOKEN_ADDRESS 
  OR dates_x_tokens.token = prices.SYMBOL
  )
AND dates_x_tokens.date_day = prices.hour::date
GROUP BY (
  dates_x_tokens.date_day,
  dates_x_tokens.symbol,
  dates_x_tokens.token,
  dates_x_tokens.pool_address,
  dates_x_tokens.pool_name
  )
ORDER BY date_day DESC

),


balances as (
select

dates.date_day,
dates.pool_address,
dates.pool_name, 
dates.token,
dates.symbol,
sum(deltas.BAL_DELTA) as balance,
(balance * dates.daily_price) as balance_usd 

from dates_x_tokens_x_price as dates
left join ethereum.core.ez_balance_deltas as deltas
ON deltas.BLOCK_TIMESTAMP::date <= dates.date_day
AND deltas.USER_ADDRESS = dates.pool_address
AND (deltas.CONTRACT_ADDRESS = dates.token
OR dates.symbol = deltas.SYMBOL)
GROUP BY (
  dates.date_day,
  dates.pool_address,
  dates.pool_name, 
  dates.token,
  dates.symbol,
  dates.daily_price
  )
ORDER BY date_day DESC

) 


SELECT
  date_day, 
  pool_name,
  sum(balance) as current_bal,
  sum(balance_usd) as current_bal_usd,
  LISTAGG(symbol, ', ') as tradeable_assets,
  pool_address
FROM balances
GROUP BY date_day, pool_address, pool_name
ORDER BY date_day DESC

"""

In [ ]:
df_liquidity_general = query_and_save(curve_liquidity_query, filename, df)